<a href="https://colab.research.google.com/github/juhumkwon/source_code/blob/main/WordEmbedding_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

import tensorflow as tf
import numpy as np

# 하이퍼파라미터
vocab_size = 1000     # 단어 집합 크기
max_len = 10          # 문장 최대 길이
d_model = 64          # 임베딩 차원
batch_size = 2

# 예제 입력 (batch_size, seq_len)
sample_input = tf.constant([
    [1, 5, 23, 42, 0, 0, 0, 0, 0, 0],    # PAD 포함 예제
    [7, 2, 15, 19, 31, 44, 58, 0, 0, 0]
])

# 워드 임베딩
embedding_layer = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=d_model)
word_embeddings = embedding_layer(sample_input)  # (batch_size, seq_len, d_model)

# 포지셔널 인코딩 함수
def get_positional_encoding(seq_len, d_model):
    pos = np.arange(seq_len)[:, np.newaxis]
    i = np.arange(d_model)[np.newaxis, :]
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    angle_rads = pos * angle_rates

    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]
    return tf.cast(pos_encoding, dtype=tf.float32)

# 포지셔널 인코딩 추가
pos_encoding = get_positional_encoding(max_len, d_model)
embedded_input = word_embeddings + pos_encoding[:, :max_len, :]  # (batch_size, seq_len, d_model)

# 간단한 멀티헤드 어텐션 (Self-Attention)
attention_layer = tf.keras.layers.MultiHeadAttention(num_heads=4, key_dim=d_model//4)
attention_output = attention_layer(query=embedded_input, value=embedded_input, key=embedded_input)

print("입력 임베딩 + 포지셔널 인코딩 shape:", embedded_input.shape)
print("Self-Attention 출력 shape:", attention_output.shape)

입력 임베딩 + 포지셔널 인코딩 shape: (2, 10, 64)
Self-Attention 출력 shape: (2, 10, 64)
